## ex08 - Jointure entre tables

Lorsque nous concevons un système de base de données complet en utilisant de bons principes de conception tels que la normalisation, différents aspects des informations doivent être séparés en tables normalisées. 

Dans un tel cas, nous avons souvent besoin de ***jointures*** pour récupérer des données de plusieurs tables dans une seule requête SELECT. 

Deux tables peuvent être jointes par un seul opérateur de jointure, mais le résultat peut être joint à nouveau avec d'autres tables. 

Il doit exister une colonne identique ou similaire entre les tables jointes.

Pour connecter des tables dans une requête, nous utilisons une instruction ***JOIN ... ON***. 

Il existe différents types de jointures SQLite :
- INNER JOIN (ou parfois appelé jointure simple)
- LEFT OUTER JOIN (ou parfois appelé LEFT JOIN)
- JOINTURE CROISÉE

La jointure de type RIGHT OUTER JOIN et FULL OUTER JOIN ne sont pas prises en charge dans SQLite.

In [ ]:
%load_ext sql

In [ ]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


### 1. Connection à la database demo.db3

In [ ]:
%sql sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3

'Connected: @/content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3'

Si vous ne vous souvenez pas des tables présentes dans la database de démonstration, vous pouvez toujours utiliser la commande suivante pour les retrouver.

In [ ]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


name
rch
hru
sub
sed
watershed_daily
watershed_monthly
watershed_yearly
channel_dimension
hru_info
sub_info


### 2. INNER JOIN

L'INNER JOIN nous permet de fusionner deux tables ensemble. Mais si nous allons fusionner des tables, nous devons définir un point commun entre les deux afin que les enregistrements des deux tables s'alignent. Nous devons définir un ou plusieurs champs qu'ils ont en commun et les rejoindre.

#### 2.1 Vérification des colonnes en commun entre deux tables

Ici, nous prenons les tableaux de ***rch*** et ***sub*** comme exemple. 
Il y a trois colonnes communes de RCH / SUB, YR et MO.


In [ ]:
%sql SELECT * From rch LIMIT 3

 * sqlite:///data/demo.db3
Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,EVAPcms,TLOSScms,SED_INtons,SED_OUTtons,SEDCONCmg_kg,ORGN_INkg,ORGN_OUTkg,ORGP_INkg,ORGP_OUTkg,NO3_INkg,NO3_OUTkg,NH4_INkg,NH4_OUTkg,NO2_INkg,NO2_OUTkg,MINP_INkg,MINP_OUTkg,CHLA_INkg,CHLA_OUTkg,CBOD_INkg,CBOD_OUTkg,DISOX_INkg,DISOX_OUTkg,SOLPST_INmg,SOLPST_OUTmg,SORPST_INmg,SORPST_OUTmg,REACTPSTmg,VOLPSTmg,SETTLPSTmg,RESUSP_PSTmg,DIFFUSEPSTmg,REACBEDPSTmg,BURYPSTmg,BED_PSTmg,BACTP_OUTct,BACTLP_OUTct,CMETAL_1kg,CMETAL_2kg,CMETAL_3kg,TOT_Nkg,TOT_Pkg,NO3ConcMg_l,WTMPdegc
1,1981,1,146.343765259,146.252487183,0.0912808850408,0.0,2.33204616507e-07,61619.4648438,155.371902466,0.0160862877965,0.0,0.0482588782907,0.0,362.048675537,361.81350708,203.620849609,421.183776855,0.0,23.0184326172,0.0161072444171,0.0,1.18390523079e-11,0.0,0.0,0.0,5627225.0,5623486.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,806.01574707,0.0,0.0,0.0
2,1981,1,96.225692749,96.1828536987,0.0428212843835,0.0,1.64267646596e-07,0.0,0.0,0.0136315366253,0.0,0.0408946201205,0.0,315.600524902,315.457977295,0.0,127.005020142,0.0,0.0,0.0136560499668,0.0,4.13697217716e-16,0.0,0.0,0.0,3757606.5,3698301.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,442.463012695,0.0,0.0,0.0
3,1981,1,11.9527187347,11.8613681793,0.0913518294692,0.0,2.03258238685e-07,2.03258238685e-07,6.59506094181e-09,0.0114662880078,0.0,0.0343988612294,0.00911803822964,48.2963752747,47.9315032959,0.0,62.4676208496,0.0,0.0,0.0114851053804,0.0,5.9410287833e-14,0.0,0.0,0.0,360979.90625,456115.90625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.399124146,0.00911803822964,0.0,0.0


In [ ]:
%sql SELECT * From sub LIMIT 3

 * sqlite:///data/demo.db3
Done.


SUB,YR,MO,PRECIPmm,SNOMELTmm,PETmm,ETmm,SWmm,PERCmm,SURQmm,GW_Qmm,WYLDmm,SYLDt_ha,ORGNkg_ha,ORGPkg_ha,NSURQkg_ha,SOLPkg_ha,SEDPkg_ha,LAT_Q_mm,LATNO3kg_h,GWNO3kg_ha,CHOLAmic_L,CBODU_mg_L,DOXQ_mg_L,TNO3kg_ha,QTILEmm,TVAPkg_ha
1,1981,1,35.6019897461,0.0,3.72074365616,0.24964235723,10.7985944748,0.0,2.37951117015e-05,0.0537296123803,0.298895508051,3.44979361917e-13,2.37964208338e-08,2.37964208338e-08,2.37964208338e-08,2.37964208338e-08,4.75928416677e-08,0.00781186530367,0.0,1.56228736614e-07,0.0,0.0,0.0,0.0,0.0,0.0
2,1981,1,108.606071472,0.0,3.45040774345,0.457205563784,56.3250045776,0.0,1.72378986463e-05,28.5720500946,32.5934295654,2.0773781187e-13,1.72388556763e-08,1.72388556763e-08,1.72388556763e-08,1.72388556763e-08,3.44777113526e-08,0.0339314937592,1.62209616974e-05,0.000382879807148,0.0,0.0,0.0,0.0,0.0,0.0
3,1981,1,149.308364868,0.0,10.566324234,6.0271062851,71.0020980835,0.0,2.4236529498e-05,3.96275544167,5.27445697784,3.34875817397e-13,1.88911428722e-08,1.88911428722e-08,1.88911428722e-08,1.88911428722e-08,3.77822857445e-08,0.00791492685676,3.72241470359e-06,7.58288224461e-05,0.0,0.0,0.0,0.0,0.0,0.0


#### 2.2 Make an inner join

The syntax for the INNER JOIN in SQLite is:
>SELECT columns<br>
>FROM table1<br> 
>INNER JOIN table2<br>
>ON table1.column = table2.column;

Join on RCH/SUB, YR and MO. 

***Note***: When SELECTing the common columns, have to clearly asign a table'name. If column names or table names are too long, we can use aliases to give them short names.

In [ ]:
%%sql sqlite://
SELECT RCH, rch.YR, rch.MO, FLOW_INcms, FLOW_OUTcms, PRECIPmm, PETmm
FROM rch INNER JOIN sub
ON rch.RCH = sub.SUB and rch.YR=sub.YR and rch.MO=sub.MO
LIMIT 5

Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,PRECIPmm,PETmm
1,1981,1,146.343765259,146.252487183,35.6019897461,3.72074365616
2,1981,1,96.225692749,96.1828536987,108.606071472,3.45040774345
3,1981,1,11.9527187347,11.8613681793,149.308364868,10.566324234
4,1981,1,49.486492157,49.4065132141,108.606048584,10.674993515
5,1981,1,274.066802979,272.106018066,201.311279297,27.1792430878


### 3. LEFT JOIN

Similar to the INNER JOIN clause, the LEFT JOIN clause is an optional clause of the SELECT statement. You use the LEFT JOIN clause to query data from multiple correlated tables. This type of join returns all rows from the LEFT-hand table specified in the ON condition and only those rows from the other table where the joined fields are equal (join condition is met).

The syntax for the SQLite LEFT OUTER JOIN is:
>SELECT columns<br>
>FROM table1<br>
>LEFT [OUTER] JOIN table2<br>
>ON table1.column = table2.column;

In [ ]:
%%sql sqlite://
SELECT RCH, rch.YR, rch.MO, FLOW_INcms, FLOW_OUTcms, PRECIPmm, PETmm
FROM rch LEFT JOIN sub
ON rch.RCH = sub.SUB and rch.YR=sub.YR and rch.MO=sub.MO
LIMIT 5

Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,PRECIPmm,PETmm
1,1981,1,146.343765259,146.252487183,35.6019897461,3.72074365616
2,1981,1,96.225692749,96.1828536987,108.606071472,3.45040774345
3,1981,1,11.9527187347,11.8613681793,149.308364868,10.566324234
4,1981,1,49.486492157,49.4065132141,108.606048584,10.674993515
5,1981,1,274.066802979,272.106018066,201.311279297,27.1792430878


### 4. CROSS JOIN

Another type of join is called a SQLite CROSS JOIN. This type of join returns a combined result set with every row from the first table matched with every row from the second table. This is also called a Cartesian Product.

The syntax for the SQLite CROSS JOIN is:
>SELECT columns<br>
>FROM table1<br>
>CROSS JOIN table2;

In [ ]:
%%sql sqlite://
SELECT RCH, rch.YR, rch.MO, FLOW_INcms, FLOW_OUTcms, PRECIPmm, PETmm
FROM rch
CROSS JOIN sub
LIMIT 5

Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,PRECIPmm,PETmm
1,1981,1,146.343765259,146.252487183,35.6019897461,3.72074365616
1,1981,1,146.343765259,146.252487183,108.606071472,3.45040774345
1,1981,1,146.343765259,146.252487183,149.308364868,10.566324234
1,1981,1,146.343765259,146.252487183,108.606048584,10.674993515
1,1981,1,146.343765259,146.252487183,201.311279297,27.1792430878


### 5. Querying Multiple Tables Using JOIN

Relational databases can be fairly complex in terms of relationships between tables. Sometimes, we have to require information from more than two tables.

We can use the following syntax to join multiple tables:
>SELECT columns<br>
>FROM table1<br>
>INNER JOIN table2 ON table1.column = table2.column<br>
>INNER JOIN table3 ON table1.column = table3.column<br>
>...<br>
>INNER JOIN tablen ON table1.column = tablen.column;

There is no limit of maximum number of tables you can join according to SQL itself. However, most DBMSes have their own limits. You should check your DBMSes docs in practical applications. In addition, the query will will slow down considerably when joining too many tables (e.g., 4 or more tables). 

### Summary

In this notebook, we practices the three major join types in SQLite: INNER, LEFT and CROSS joins. Joins allow us to take data scattered across multiple tables and stitch it together into something more meaningful and descriptive. We can take two or more tables and join them together into a larger table that has more context. Moreover, using aliases enables us to rename column or table names on the fly.